In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.io
import plotly.express as px
from scipy import ndimage as ndi
import imageio
from scipy import signal
from pathlib import Path
import tensorflow_probability as tfp
import tensorflow as tf
from tqdm import tqdm  
from os import path
from myfunctions import event_separation , image_crop_save_gauss , image_crop_save , image_crop_negative, poi
import tifffile


## Input data from a folder ##

In [5]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\original_data'
images_dir = '220927_mitotrackergreen_cos7_ZEISS_fl'
joined_path = os.path.join(files_dir, images_dir)
size=(2048,2048)
img,input_name,output_name,datacsv={},{},{},{}

for input_file in os.listdir(joined_path):
        joined_file_path = os.path.join(files_dir, images_dir, input_file)
        if '.tif' in input_file:
            date, cell_type, dye, bf_fl, index  = input_file.split('_')
            number, ome, tiff= index.split('.')
            img[int(number)-1] = Image.open(joined_file_path)
            input_name[int(number)-1]= f'points_{date}_{cell_type}_{bf_fl}_{number}'
            output_name[int(number)-1]= f'image_{date}_{cell_type}_{bf_fl}_{number}'
            print('Loaded image:'+input_file)
        else:
            labels, index, bf_fl  = input_file.split('_')
            datacsv[int(index)-1] = pd.read_csv(joined_file_path)
            print('Loaded csv:'+ input_file)
max_number=int(number)        


Loaded image:220923_cos7_mitotrackergreen_GFP_1.ome.tif
Loaded image:220923_cos7_mitotrackergreen_GFP_2.ome.tif
Loaded image:220923_cos7_mitotrackergreen_GFP_3.ome.tif
Loaded image:220923_cos7_mitotrackergreen_GFP_4.ome.tif
Loaded image:220923_cos7_mitotrackergreen_GFP_5.ome.tif
Loaded image:220923_cos7_mitotrackergreen_GFP_6.ome.tif
Loaded csv:labels_1_f.csv
Loaded csv:labels_2_f.csv
Loaded csv:labels_3_f.csv
Loaded csv:labels_4_f.csv
Loaded csv:labels_5_f.csv
Loaded csv:labels_6_f.csv


## Get all Gaussian Points of Interest ##

In [6]:
for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    for i in range(1,10,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        framenum=im.n_frames
        poi(csv,in_name,sigma,size,framenum)

100%|██████████| 51/51 [00:12<00:00,  4.11it/s]


## Cut Images and Gaussian POIs ##

In [12]:
base_dir= r'C:\Users\roumba\Documents\Software\deep-events'
training_dir= r'C:\Users\roumba\Documents\Software\deep-events\training_data'
folder_name=images_dir+'_pos'
filepath= os.path.join(training_dir , folder_name)

for x in range(2,max_number):
    csv=datacsv[x]
    im=img[x]
    for i in range(1,10,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        out_name=f'{output_name[x]}'
        out_name_g=f'{output_name[x]}_sigma{s}'
    
        list1=event_separation(csv)
        l=len(list1)
        image_crop_save(l,list1, csv, im, out_name,filepath)

        gauss=base_dir+f'\{in_name}'
        gauss_image=Image.open(gauss)
        image_crop_save_gauss(l,list1, csv, gauss_image, out_name_g,filepath)

In [21]:
csv=datacsv[1]
im=img[1]
for i in range(9,10,1):
    sigma=(i,i)
    s=sigma[0]
    in_name=f'{input_name[x]}_sigma{s}.tiff'
    out_name=f'{output_name[x]}'
    out_name_g=f'{output_name[x]}_sigma{s}'

    gauss=base_dir+f'\{in_name}'
    gauss_image=Image.open(gauss)
    image_crop_save_gauss(l,list1, csv, gauss_image, out_name_g,filepath)
    
    list1=event_separation(csv)
    l=len(list1)
    image_crop_save(l,list1, csv, im, out_name,filepath)

    

ValueError: cannot convert float NaN to integer

## Cut Negative Frames ##

In [ ]:
folder_name=images_dir+'_neg'
filepath= os.path.join(training_dir , folder_name)

for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    out_name=output_name[x]
    in_name=input_name[x] 
    image_crop_negative(l,list1, csv, im, out_name,filepath)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\roumba\\Documents\\Software\\deep-events\\training_data\\220922_mtstaygold_U2OS_iSIM_fl_neg\\image_220922_mtstaygold_fl_1_0_neg.tiff'